In [3]:
import pandas as pd
from rdflib import Graph, URIRef, Literal, RDF, RDFS, Namespace, BNode
from urllib.parse import quote
from rdflib.plugins.sparql import prepareQuery
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
from itertools import islice
from fuzzywuzzy import fuzz


In [4]:
pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install fuzzywuzzy[speedup]

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: fuzzywuzzy[speedup] in c:\users\56946\.pyenv\pyenv-win\versions\3.10.10\lib\site-packages (0.18.0)




[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
#CONSULTA OBTENER MINISTERIOS EN WIKIDATA
Ministerios_dic={}
from SPARQLWrapper import SPARQLWrapper, JSON
# Definir el punto de acceso SPARQL de Wikidata
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
# Definir la consulta SPARQL
sparql.setQuery("""
SELECT ?ministerio ?ministerioLabel
WHERE {
  ?ministerio wdt:P31 wd:Q55593474.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],es". }
}
""")
# Establecer el formato de los resultados
sparql.setReturnFormat(JSON)
# Ejecutar la consulta y procesar los resultados
results = sparql.query().convert()
i=0
for result in results["results"]["bindings"]:
    Ministerios_dic[result["ministerioLabel"]["value"]]=result["ministerio"]["value"]
    print(result["ministerioLabel"]["value"],result["ministerio"]["value"])

Ministerio de Educación de Chile http://www.wikidata.org/entity/Q3079732
Ministerio del Interior y Seguridad Pública de Chile http://www.wikidata.org/entity/Q4294320
Ministerio de Minería de Chile http://www.wikidata.org/entity/Q4294331
Ministerio de Salud de Chile http://www.wikidata.org/entity/Q4294370
Ministerio de Defensa Nacional de Chile http://www.wikidata.org/entity/Q4294482
Ministerio de Obras Públicas de Chile http://www.wikidata.org/entity/Q4294544
Ministerio de Transportes y Telecomunicaciones de Chile http://www.wikidata.org/entity/Q4294749
Ministerio de Hacienda de Chile http://www.wikidata.org/entity/Q4294804
Ministerio de Economía, Fomento y Turismo de Chile http://www.wikidata.org/entity/Q4294819
Ministerio de Relaciones Exteriores de Chile http://www.wikidata.org/entity/Q4294926
Ministerio de Energía de Chile http://www.wikidata.org/entity/Q5549812
Ministerio de Agricultura de Chile http://www.wikidata.org/entity/Q6017174
Ministerio de Bienes Nacionales de Chile http:

In [9]:
#CONSULTA OBTENER MINISTERIOS EN WIKIDATA CON DIRECCIÓN
Ministerios_direc_dic={}
from SPARQLWrapper import SPARQLWrapper, JSON
# Definir el punto de acceso SPARQL de Wikidata
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
# Definir la consulta SPARQL
sparql.setQuery("""
SELECT ?ministerio ?direccion ?ministerioLabel ?direccionLabel
WHERE {
  ?ministerio wdt:P31 wd:Q55593474.
  ?ministerio wdt:P159 ?direccion.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],es". }
}
""")
# Establecer el formato de los resultados
sparql.setReturnFormat(JSON)
# Ejecutar la consulta y procesar los resultados
results = sparql.query().convert()
i=0
for result in results["results"]["bindings"]:
    direccion = result["direccionLabel"]["value"]
    Ministerios_direc_dic[result["ministerioLabel"]["value"]]=[result["ministerio"]["value"],direccion]
    print(result["ministerio"]["value"],result["ministerioLabel"]["value"],direccion)

http://www.wikidata.org/entity/Q3079732 Ministerio de Educación de Chile Comuna de Santiago
http://www.wikidata.org/entity/Q4294370 Ministerio de Salud de Chile Comuna de Santiago
http://www.wikidata.org/entity/Q4294544 Ministerio de Obras Públicas de Chile Comuna de Santiago
http://www.wikidata.org/entity/Q4294804 Ministerio de Hacienda de Chile Comuna de Santiago
http://www.wikidata.org/entity/Q4294926 Ministerio de Relaciones Exteriores de Chile Comuna de Santiago
http://www.wikidata.org/entity/Q5549812 Ministerio de Energía de Chile Comuna de Santiago
http://www.wikidata.org/entity/Q6017243 Ministerio de Desarrollo Social y Familia de Chile Comuna de Santiago
http://www.wikidata.org/entity/Q6017368 Ministerio de Justicia y Derechos Humanos de Chile Comuna de Santiago
http://www.wikidata.org/entity/Q6017498 Ministerio de Vivienda y Urbanismo de Chile Comuna de Santiago
http://www.wikidata.org/entity/Q6017552 Ministerio del Medio Ambiente de Chile Comuna de Santiago
http://www.wikida

In [10]:
#CONSULTA OBTENER INSTITUCIONES EN WIKIDATA
Instituciones_dic={}
from SPARQLWrapper import SPARQLWrapper, JSON
# Definir el punto de acceso SPARQL de Wikidata
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
# Definir la consulta SPARQL
sparql.setQuery("""
SELECT ?institucion ?institucionLabel
WHERE {
  ?institucion wdt:P31 wd:Q327333.
  ?institucion wdt:P17 wd:Q298.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],es". }
}
""")
# Establecer el formato de los resultados
sparql.setReturnFormat(JSON)
# Ejecutar la consulta y procesar los resultados
results = sparql.query().convert()
i=0
j=0

for result in results["results"]["bindings"]:
    i+=1
    Instituciones_dic[result["institucionLabel"]["value"]]=result["institucion"]["value"]
print(i)

92


In [11]:
Institu_direc={}
#RETORNA LAS INSTITUCIONES JUNTO A SU DIRECCION
from SPARQLWrapper import SPARQLWrapper, JSON

# Definir el punto de acceso SPARQL de Wikidata
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# Definir la consulta SPARQL para obtener las instituciones de Chile y sus direcciones
sparql.setQuery("""
SELECT ?institucion ?institucionLabel ?direccion ?direccionLabel
WHERE {
  ?institucion wdt:P31 wd:Q327333.
  ?institucion wdt:P17 wd:Q298.
  ?institucion wdt:P159 ?direccion.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],es". }
}
""")

# Establecer el formato de los resultados
sparql.setReturnFormat(JSON)

# Ejecutar la consulta y procesar los resultados
results = sparql.query().convert()
i=0
for result in results["results"]["bindings"]:
    institucion = result["institucionLabel"]["value"]
    direccion = result["direccionLabel"]["value"]
    iq=result["institucion"]["value"]
    Institu_direc[institucion]=[iq,direccion]
    print(f"Institución: {institucion} - Dirección: {direccion}")

Institución: Instituto Antártico Chileno - Dirección: Punta Arenas
Institución: Instituto Nacional de Deportes - Dirección: Providencia
Institución: Consejo de Seguridad Nacional de Chile - Dirección: Palacio de La Moneda
Institución: Comisión Chilena del Cobre - Dirección: Comuna de Santiago
Institución: Consejo de Monumentos Nacionales - Dirección: Avenida Vicuña Mackenna
Institución: Consejo para la Transparencia - Dirección: Santiago de Chile
Institución: Corporación Nacional de Desarrollo Indígena - Dirección: Temuco
Institución: Dirección Meteorológica de Chile - Dirección: Avenida Portales
Institución: Dirección Nacional de Fronteras y Límites del Estado - Dirección: Comuna de Santiago
Institución: El Fondo Nacional de Desarrollo Científico y Tecnológico - Dirección: Santiago de Chile
Institución: Fondo Nacional de Desarrollo Cultural y las Artes - Dirección: Plaza Sotomayor
Institución: Instituto Forestal de Chile - Dirección: Santiago de Chile
Institución: Instituto Nacional d

## Cargar excel al grafo

### Instituciones en wikidata

In [12]:
dfcert = pd.read_excel('tramites/certificadosnormsinespaciocertlisto.xlsx')
dfdoc = pd.read_excel('tramites/certificadosnormsinespaciodoclisto.xlsx')
dftrans = pd.read_excel('tramites/transacciones2023.xlsx')

In [13]:
#Se crean estos conjuntos para identificar los elementos del excel que están en wikidata

institucion_unicos = dfcert['Institución'].unique() #Ministerios que estan en el excel
institucion_no_wiki=institucion_unicos.copy().tolist()#Elementos del excel que no están en wikidata
institu_interseccion=[]

llaves_institu_wikidata=Instituciones_dic.keys()
llaves_institu_no_excel=[]
for copi in llaves_institu_wikidata:
    llaves_institu_no_excel.append(copi)

for mini in institucion_unicos:
    if mini in Instituciones_dic: #Si el ministerio se encuentra en el diccionario, lo identifico
        institucion_no_wiki.remove(mini)
        llaves_institu_no_excel.remove(mini)
        institu_interseccion.append(mini)

In [ ]:
print((llaves_institu_no_excel))

In [ ]:
print(institucion_no_wiki)

In [14]:
similitud_instituciones={}
for institucion in institucion_no_wiki:
    best=['',0]
    for candidato in llaves_institu_no_excel:
        comparacion =fuzz.ratio(institucion, candidato)
        if comparacion > best[1]:
            best[0]=candidato
            best[1]=comparacion
    similitud_instituciones[institucion]=best

In [15]:
diccionario_ordenado = dict(sorted(similitud_instituciones.items(), key=lambda item: item[1][1], reverse=True))

In [16]:
for prin in diccionario_ordenado:
    print(prin,diccionario_ordenado[prin])

Servicio de Salud Viña del Mar - Quillota ['Servicio de Salud Viña del Mar-Quillota', 98]
Servicio de Salud Maule ['Servicio de Salud del Maule', 92]
Agencia Nacional de Investigación y Desarrollo ['Agencia Nacional de Investigación y Desarrollo de Chile', 91]
Dirección General del Territorio Marítimo y de Marina Mercante ['Dirección General del Territorio Marítimo y Marina Mercante de Chile', 91]
Servicio Nacional de Geología y Minería ['Servicio Nacional de Geología y Minería de Chile', 90]
Contraloría General de la República ['Contraloría General de la República de Chile', 89]
Subsecretaría de Educación Superior ['Subsecretaría de Educación Superior de Chile', 89]
Servicio de Evaluación Ambiental ['Servicio de Evaluación Ambiental de Chile', 88]
Instituto Geográfico Militar ['Instituto Geográfico Militar (Chile)', 88]
Instituto Nacional de Deportes de Chile ['Instituto Nacional de Deportes', 87]
Dirección de Presupuestos ['Dirección de Presupuestos de Chile', 85]
Defensoría Penal Pú

In [17]:
#Recuperando los primeros 15 elementos que tienen más similitud
primeros_15_elementos = dict(islice(diccionario_ordenado.items(), 15))

In [18]:
del primeros_15_elementos['Servicio Local de Educación Pública de Huasco']
del primeros_15_elementos['Servicio de Salud Ñuble']

In [19]:
for prin in primeros_15_elementos:
    print(primeros_15_elementos[prin],prin)

['Servicio de Salud Viña del Mar-Quillota', 98] Servicio de Salud Viña del Mar - Quillota
['Servicio de Salud del Maule', 92] Servicio de Salud Maule
['Agencia Nacional de Investigación y Desarrollo de Chile', 91] Agencia Nacional de Investigación y Desarrollo
['Dirección General del Territorio Marítimo y Marina Mercante de Chile', 91] Dirección General del Territorio Marítimo y de Marina Mercante
['Servicio Nacional de Geología y Minería de Chile', 90] Servicio Nacional de Geología y Minería
['Contraloría General de la República de Chile', 89] Contraloría General de la República
['Subsecretaría de Educación Superior de Chile', 89] Subsecretaría de Educación Superior
['Servicio de Evaluación Ambiental de Chile', 88] Servicio de Evaluación Ambiental
['Instituto Geográfico Militar (Chile)', 88] Instituto Geográfico Militar
['Instituto Nacional de Deportes', 87] Instituto Nacional de Deportes de Chile
['Dirección de Presupuestos de Chile', 85] Dirección de Presupuestos
['Defensoría Penal 

In [20]:
#typo_con_direc=['Contraloría General de la República','Instituto Nacional de Deportes de Chile']
#excepcion=['Ministerio de las Culturas, las Artes y el Patrimonio','Ministerio de Ciencia, Tecnología, Conocimiento e Innovación']
instituciones_unicos = dfcert['Institución'].unique() #Ministerios que estan en el excel
instituciones_wikidata={}
for insti in instituciones_unicos:
    if insti in primeros_15_elementos:
        ins=primeros_15_elementos[insti][0]
        if ins in Institu_direc:
            b=Institu_direc[ins]
            instituciones_wikidata[insti]=[b[0],b[1]]
        else:
            b=Instituciones_dic[ins]
            instituciones_wikidata[insti]=[b]
    elif insti in institucion_no_wiki:
        continue
    else: 
        if insti in Institu_direc:
            b=Institu_direc[insti]
            instituciones_wikidata[insti]=[b[0],b[1]]
        else:
            a=Instituciones_dic[insti]
            instituciones_wikidata[insti]=[a]

In [21]:
for j in instituciones_wikidata:
    print(j,instituciones_wikidata[j])

Servicio Nacional de Aduanas ['http://www.wikidata.org/entity/Q18397831']
Corporación Nacional de Desarrollo Indígena ['http://www.wikidata.org/entity/Q5788931', 'Temuco']
Dirección General de Obras Públicas ['http://www.wikidata.org/entity/Q106190177']
Dirección General del Crédito Prendario ['http://www.wikidata.org/entity/Q5808001']
Dirección General de Aguas ['http://www.wikidata.org/entity/Q106191319']
Dirección del Trabajo ['http://www.wikidata.org/entity/Q5808075']
Fondo de Solidaridad e Inversión Social ['http://www.wikidata.org/entity/Q16568463', 'Santiago de Chile']
Instituto Antártico Chileno ['http://www.wikidata.org/entity/Q119009', 'Punta Arenas']
Instituto Nacional de Propiedad Industrial ['http://www.wikidata.org/entity/Q5917578', 'Santiago de Chile']
Instituto Nacional de Deportes de Chile ['http://www.wikidata.org/entity/Q1072817', 'Providencia']
Instituto de Desarrollo Agropecuario ['http://www.wikidata.org/entity/Q5918097']
Junta de Aeronáutica Civil ['http://www.wi

### Ministerios wikidata

In [22]:
ministerios_unicos = dfcert['Ministerio'].unique() #Ministerios que estan en el excel
ministerios_formato_wikidata=[] #Ministerios que estan en el excel en formato wikidata
excepcion=['Ministerio de las Culturas, las Artes y el Patrimonio','Ministerio de Ciencia, Tecnología, Conocimiento e Innovación']

for mini in ministerios_unicos:
    if mini in excepcion:
        ministerios_formato_wikidata.append(mini)
    else:
        ministerios_formato_wikidata.append(mini + ' de Chile')
ministerios_formato_wikidata_no_wiki=ministerios_formato_wikidata.copy()#Elementos del excel que no están en wikidata

In [23]:
llaves_wikidata=Ministerios_dic.keys()
llaves_wikidata_no_excel=[]
for copi in llaves_wikidata:
    llaves_wikidata_no_excel.append(copi)
    
for mini in ministerios_formato_wikidata:
    if mini in Ministerios_dic: #Si el ministerio se encuentra en el diccionario, lo identifico
        ministerios_formato_wikidata_no_wiki.remove(mini)
        llaves_wikidata_no_excel.remove(mini)

In [ ]:
print(ministerios_formato_wikidata_no_wiki)

In [24]:
for j in Ministerios_direc_dic:
    print(j,Ministerios_direc_dic[j])

Ministerio de Educación de Chile ['http://www.wikidata.org/entity/Q3079732', 'Comuna de Santiago']
Ministerio de Salud de Chile ['http://www.wikidata.org/entity/Q4294370', 'Comuna de Santiago']
Ministerio de Obras Públicas de Chile ['http://www.wikidata.org/entity/Q4294544', 'Comuna de Santiago']
Ministerio de Hacienda de Chile ['http://www.wikidata.org/entity/Q4294804', 'Comuna de Santiago']
Ministerio de Relaciones Exteriores de Chile ['http://www.wikidata.org/entity/Q4294926', 'Comuna de Santiago']
Ministerio de Energía de Chile ['http://www.wikidata.org/entity/Q5549812', 'Comuna de Santiago']
Ministerio de Desarrollo Social y Familia de Chile ['http://www.wikidata.org/entity/Q6017243', 'Comuna de Santiago']
Ministerio de Justicia y Derechos Humanos de Chile ['http://www.wikidata.org/entity/Q6017368', 'Comuna de Santiago']
Ministerio de Vivienda y Urbanismo de Chile ['http://www.wikidata.org/entity/Q6017498', 'Comuna de Santiago']
Ministerio del Medio Ambiente de Chile ['http://www.

In [25]:
ministerios_unicos = dfcert['Ministerio'].unique() #Ministerios que estan en el excel
ministerios_wikidata={}
i=0
for mini in ministerios_unicos:
    if mini=='Ministerio Secretaria General de Gobierno de Chile':
        if 'Ministerio Secretaría General de Gobierno de Chile' in Ministerios_direc_dic:
            a=Ministerios_direc_dic['Ministerio Secretaría General de Gobierno de Chile']
            ministerios_wikidata[mini]=[a[0],a[1]]
        else:
            a=Ministerios_dic['Ministerio Secretaría General de Gobierno de Chile']
            ministerios_wikidata[mini]=[a]
            
    elif mini in excepcion:
        if mini in Ministerios_direc_dic:
            a=Ministerios_direc_dic[mini]
            ministerios_wikidata[mini]=[a[0],a[1]]
        else:
            a=Ministerios_dic[mini]
            ministerios_wikidata[mini]=[a]
    elif mini +' de Chile'in ministerios_formato_wikidata_no_wiki:
        continue
    else:
        llave= mini+ ' de Chile'
        if llave in Ministerios_direc_dic:
            a=Ministerios_direc_dic[mini+' de Chile']
            ministerios_wikidata[mini]=[a[0],a[1]]
        else:
            a=Ministerios_dic[mini+' de Chile']
            ministerios_wikidata[mini]=[a]

In [ ]:
for j in ministerios_wikidata:
    print (j,ministerios_wikidata[j])

In [ ]:
print(llaves_wikidata_no_excel)

In [ ]:
print(ministerios_formato_wikidata_no_wiki)

## Crear el arbol con nodos de wikidata

In [27]:
start_time = time.time()
# Crear el grafo y el espacio de nombres
g = Graph()
ex = Namespace("http://example.org/")
# Diccionario para realizar un seguimiento de las instituciones
Ministerios = {}
Ministerios_uri = {}
Tramitesdic = {}
# Iterar sobre el DataFrame y agregar ministerio, instituciones y tramites con documentos
for _, row in dfcert.iterrows():
    tramite_codigo = row['Código Trámite']
    tramite_nombre = row['Nombre del trámite']
    tramite_uri = Tramitesdic.get(tramite_codigo)
    #tramite_uri = Tramitesdic.get(tramite_nombre) #usar para construir en base al nombre
    if not tramite_uri:
        Ministerio_nombre = row['Ministerio']
        Ministerio_uri = Ministerios_uri.get(Ministerio_nombre)
        # Si el ministerio no existe en el diccionario, crear el nodo
        if not Ministerio_uri:
            if Ministerio_nombre in ministerios_wikidata:
                Ministerio_uri = URIRef(ministerios_wikidata[Ministerio_nombre][0])
                g.add((Ministerio_uri, RDF.type, ex.Ministerio))
                Ministerios_uri[Ministerio_nombre] = Ministerio_uri
                Ministerios[Ministerio_nombre]={}
                aja=Ministerio_nombre
                nombre = ex[quote(aja.replace(" ", "_"))]
                g.add((Ministerio_uri, RDFS.label, Literal(aja)))
                if len(ministerios_wikidata[Ministerio_nombre])==2:
                    dire=ministerios_wikidata[Ministerio_nombre][1]
                    g.add((Ministerio_uri, ex.direccion,Literal(dire)))
            else:
                Ministerio_uri = ex[quote(Ministerio_nombre.replace(" ", "_"))]
                g.add((Ministerio_uri, RDF.type, ex.Ministerioblanc))
                Ministerios_uri[Ministerio_nombre] = Ministerio_uri
                Ministerios[Ministerio_nombre]={}
        institucion_nombre = row['Institución']
        institucion_uri = Ministerios[Ministerio_nombre].get(institucion_nombre)
        # Si la institución no existe en el diccionario, crear el nodo
        if not institucion_uri:
            if institucion_nombre in instituciones_wikidata:   
                institucion_uri = URIRef(instituciones_wikidata[institucion_nombre][0])
                g.add((institucion_uri, RDF.type, ex.Institucion))
                g.add((Ministerio_uri, ex.tieneInstitucion, institucion_uri))
                g.add((institucion_uri, RDFS.label,Literal(institucion_nombre)))
                Ministerios[Ministerio_nombre][institucion_nombre] = institucion_uri
                
                if len(instituciones_wikidata[institucion_nombre])==2:
                    dire=instituciones_wikidata[institucion_nombre][1]
                    g.add((institucion_uri, ex.direccion, Literal(dire)))
            else:
                institucion_uri = ex[quote(institucion_nombre.replace(" ", "_"))]
                g.add((institucion_uri, RDF.type, ex.Institucionblanc))
                g.add((Ministerio_uri, ex.tieneInstitucion, institucion_uri))
                Ministerios[Ministerio_nombre][institucion_nombre] = institucion_uri
        # Crear el nodo para el tramite y conectarlo a la institución
        tramite_nombre = row['Nombre del trámite']
        tramite_codigo = row['Código Trámite']
        tramite_uri = ex[quote(tramite_codigo.replace(" ", "_"))]
        g.add((tramite_uri, RDF.type, ex.Tramite))
        g.add((institucion_uri, ex.realizaTramite, tramite_uri))
        Tramitesdic[tramite_codigo]=tramite_uri
    if row['¿Requiere de algún certificado?'] == 'Sí':
        # Crear nodos para cada documento y conectarlos al tramite
        certificado_nombre = row['Detalle certificados']
        certificado_uri = ex[quote(certificado_nombre.replace(" ", "_"))]
        g.add((certificado_uri, RDF.type, ex.certificado))
        g.add((tramite_uri, ex.requiereCertificado, certificado_uri))
#-------------------------Requerimientos-----------------

#-------Documentos notariales----------------------------
    # Verificar si el tramite requiere documento
for _, row in dfdoc.iterrows():
    tramite_codigo = row['Código Trámite']
    tramite_nombre = row['Nombre del trámite']
    tramite_uri = Tramitesdic.get(tramite_codigo)
    if row['¿Requiere de un documento notarial?'] == 'Sí':
        documento_nombre = row['Detalle documentos']
        documento_uri = ex[quote(documento_nombre.replace(" ","_"))]
        g.add((documento_uri, RDF.type, ex.Documento))
        g.add((tramite_uri, ex.requiereDocumento, documento_uri))
end_time = time.time()
elapsed_time = end_time - start_time

In [28]:
g.serialize(destination='arbol.rdf', format='xml')

<Graph identifier=Nab1500d59b2e4956814ed194b1986e08 (<class 'rdflib.graph.Graph'>)>

In [30]:
graf = Graph()

# Carga el grafo desde un archivo RDF/XML
graf.parse('arbol.rdf', format='xml')


<Graph identifier=Na22ee576930b48baaa234d104b6872f8 (<class 'rdflib.graph.Graph'>)>

# Consultas

In [31]:

query_instituciones_salud = """
        PREFIX ex: <http://example.org/>

        SELECT DISTINCT ?ministerio
        WHERE {
            ?ministerio rdf:type ex:Ministerioblanc ;
        }
    """
    # Ejecutar la consulta en el grafo RDF
result_instituciones_salud = graf.query(query_instituciones_salud)
    # Imprimir los resultados
#    print("Instituciones asociadas al Ministerio de Salud:")
print('Ministerios que no están en wikidata')
for row in result_instituciones_salud:
    print(row.ministerio)

Ministerios que no están en wikidata
http://example.org/Defensor%C3%ADa_de_los_Derechos_de_la_Ni%C3%B1ez
http://example.org/Consejo_de_Defensa_del_Estado
http://example.org/Servicio_Electoral
http://example.org/Presidencia_de_la_Rep%C3%BAblica
http://example.org/Consejo_para_la_Transparencia
http://example.org/Ministerio_Secretaria_General_de_Gobierno
http://example.org/Conservador_de_Bienes_Ra%C3%ADces
http://example.org/Contralor%C3%ADa_General_de_la_Rep%C3%BAblica


In [34]:
query_instituciones_salud = """
        PREFIX ex: <http://example.org/>

        SELECT DISTINCT ?ministerio ?nombre ?dire
        WHERE {
            ?ministerio rdf:type ex:Ministerio.
            ?ministerio rdfs:label ?nombre.
            OPTIONAL {
            ?ministerio ex:direccion ?dire.
        }        
        }
    """
    # Ejecutar la consulta en el grafo RDF
query_instituciones_salud = prepareQuery(query_instituciones_salud, initNs={"rdf": RDF, "ex": ex})

result_instituciones_salud = graf.query(query_instituciones_salud)
    # Imprimir los resultados
#    print("Instituciones asociadas al Ministerio de Salud:")
print('Ministerios que están en wikidata y tienen dirección')
for row in result_instituciones_salud:
    if row[2]==None:
        print(row.ministerio, row.nombre + ', sin localización en wikidata')
    else:
        print(row.ministerio, row.nombre + ' Se encuentra en '+row.dire)

Ministerios que están en wikidata y tienen dirección
http://www.wikidata.org/entity/Q6017582 Ministerio del Trabajo y Previsión Social Se encuentra en Calle Huérfanos
http://www.wikidata.org/entity/Q6017174 Ministerio de Agricultura, sin localización en wikidata
http://www.wikidata.org/entity/Q4294370 Ministerio de Salud Se encuentra en Comuna de Santiago
http://www.wikidata.org/entity/Q4294482 Ministerio de Defensa Nacional, sin localización en wikidata
http://www.wikidata.org/entity/Q4294331 Ministerio de Minería, sin localización en wikidata
http://www.wikidata.org/entity/Q4294804 Ministerio de Hacienda Se encuentra en Comuna de Santiago
http://www.wikidata.org/entity/Q4294819 Ministerio de Economía, Fomento y Turismo, sin localización en wikidata
http://www.wikidata.org/entity/Q6017368 Ministerio de Justicia y Derechos Humanos Se encuentra en Comuna de Santiago
http://www.wikidata.org/entity/Q4294926 Ministerio de Relaciones Exteriores Se encuentra en Comuna de Santiago
http://www.

In [33]:
query_instituciones_salud = """
        PREFIX ex: <http://example.org/>

        SELECT DISTINCT ?ministerio
        WHERE {
            ?ministerio rdf:type ex:Institucionblanc ;
        }
    """
    # Ejecutar la consulta en el grafo RDF
result_instituciones_salud = graf.query(query_instituciones_salud)
    # Imprimir los resultados
#    print("Instituciones asociadas al Ministerio de Salud:")
print('Instituciones que no están en wikidata')
for row in result_instituciones_salud:
    print(row.ministerio)

Instituciones que no están en wikidata
http://example.org/Superintendencia_de_Casinos_de_Juego
http://example.org/Superintendencia_de_Educaci%C3%B3n
http://example.org/Tesorer%C3%ADa_General_de_La_Rep%C3%BAblica
http://example.org/Subsecretar%C3%ADa_de_Transportes
http://example.org/Consejo_Nacional_de_Educaci%C3%B3n
http://example.org/Fiscal%C3%ADa_del_Ministerio_de_Obras_P%C3%BAblicas
http://example.org/Gobierno_Regional_Metropolitano_de_Santiago
http://example.org/Fuerza_A%C3%A9rea_de_Chile
http://example.org/Subsecretar%C3%ADa_del_Interior
http://example.org/Servicio_de_Salud_Metropolitano_Occidente
http://example.org/Servicio_de_Registro_Civil_e_Identificaci%C3%B3n
http://example.org/Corporaci%C3%B3n_de_Fomento_de_la_Producci%C3%B3n
http://example.org/Fiscal%C3%ADa_Nacional_Econ%C3%B3mica
http://example.org/Subsecretar%C3%ADa_de_Salud_P%C3%BAblica
http://example.org/Subsecretar%C3%ADa_de_Bienes_Nacionales
http://example.org/Gobierno_Regional_del_Maule
http://example.org/Servicio_d

In [32]:
query_instituciones_salud = """
        PREFIX ex: <http://example.org/>

        SELECT DISTINCT ?institucion ?nombre ?dire
        WHERE {
            ?institucion rdf:type ex:Institucion.
            ?institucion rdfs:label ?nombre.
            OPTIONAL {
            ?institucion ex:direccion ?dire.
        }        
        }
    """
    # Ejecutar la consulta en el grafo RDF
query_instituciones_salud = prepareQuery(query_instituciones_salud, initNs={"rdf": RDF, "ex": ex})

result_instituciones_salud = graf.query(query_instituciones_salud)
    # Imprimir los resultados
#    print("Instituciones asociadas al Ministerio de Salud:")
print('Ministerios que están en wikidata y tienen dirección')
for row in result_instituciones_salud:
    if row[2]==None:
        print(row.institucion, row.nombre + ' Sin localización en wikidata')
    else:
        print(row.institucion, row.nombre + ' Se encuentra en '+row.dire)

Ministerios que están en wikidata y tienen dirección
http://www.wikidata.org/entity/Q5801311 Defensa Civil de Chile Sin localización en wikidata
http://www.wikidata.org/entity/Q19946968 Instituto Nacional de la Juventud Se encuentra en Comuna de Santiago
http://www.wikidata.org/entity/Q18713269 Junta de Aeronáutica Civil Sin localización en wikidata
http://www.wikidata.org/entity/Q5808003 Dirección General del Territorio Marítimo y de Marina Mercante Sin localización en wikidata
http://www.wikidata.org/entity/Q5808075 Dirección del Trabajo Sin localización en wikidata
http://www.wikidata.org/entity/Q60169264 Agencia Nacional de Investigación y Desarrollo Sin localización en wikidata
http://www.wikidata.org/entity/Q6126047 Servicio Nacional de Capacitación y Empleo Sin localización en wikidata
http://www.wikidata.org/entity/Q5764465 Servicio Nacional de Geología y Minería Sin localización en wikidata
http://www.wikidata.org/entity/Q47496083 Comisión para el Mercado Financiero Se encuent

In [36]:
query_instituciones_salud = """
        PREFIX ex: <http://example.org/>

        SELECT DISTINCT ?institucion ?nombre ?dire
        WHERE {
            ?institucion rdf:type ex:Institucion.
            ?institucion rdfs:label ?nombre.
            OPTIONAL {
            ?institucion ex:direccion ?dire.
        }        
        }
    """
    # Ejecutar la consulta en el grafo RDF
query_instituciones_salud = prepareQuery(query_instituciones_salud, initNs={"rdf": RDF, "ex": ex})

result_instituciones_salud = graf.query(query_instituciones_salud)
    # Imprimir los resultados
#    print("Instituciones asociadas al Ministerio de Salud:")
print('Ministerios que están en wikidata')
for row in result_instituciones_salud:
    if row[2]==None:
        print(row.institucion, row.nombre + ', sin localización en wikidata')
    else:
        print(row.institucion, row.nombre + ' Se encuentra en '+row.dire)

Ministerios que están en wikidata
http://www.wikidata.org/entity/Q5801311 Defensa Civil de Chile, sin localización en wikidata
http://www.wikidata.org/entity/Q19946968 Instituto Nacional de la Juventud Se encuentra en Comuna de Santiago
http://www.wikidata.org/entity/Q18713269 Junta de Aeronáutica Civil, sin localización en wikidata
http://www.wikidata.org/entity/Q5808003 Dirección General del Territorio Marítimo y de Marina Mercante, sin localización en wikidata
http://www.wikidata.org/entity/Q5808075 Dirección del Trabajo, sin localización en wikidata
http://www.wikidata.org/entity/Q60169264 Agencia Nacional de Investigación y Desarrollo, sin localización en wikidata
http://www.wikidata.org/entity/Q6126047 Servicio Nacional de Capacitación y Empleo, sin localización en wikidata
http://www.wikidata.org/entity/Q5764465 Servicio Nacional de Geología y Minería, sin localización en wikidata
http://www.wikidata.org/entity/Q47496083 Comisión para el Mercado Financiero Se encuentra en Santia